In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset1/dataset_normal.csv
/kaggle/input/dataset1/dataset_attack.csv


In [22]:
pip install codecarbon

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [23]:

from codecarbon import EmissionsTracker


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns; sns.set()

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional
from keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix

number_of_samples = 100000

In [25]:



data_attack = pd.read_csv('/kaggle/input/dataset1/dataset_attack.csv', nrows = number_of_samples)

data_normal = pd.read_csv('/kaggle/input/dataset1/dataset_normal.csv', nrows = number_of_samples)

In [26]:


data_normal.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']
data_attack.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']

In [27]:

data_normal=data_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
data_attack=data_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)

features=[ 'frame.len', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta']

In [28]:

X_normal= data_normal[features].values
X_attack= data_attack[features].values
Y_normal= data_normal['class']
Y_attack= data_attack['class']
X=np.concatenate((X_normal,X_attack))
Y=np.concatenate((Y_normal,Y_attack))

scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)

In [29]:

for i in range(0,len(Y)):
  if Y[i] =="attack":
    Y[i]=0
  else:
    Y[i]=1

print(X.shape)

features = len(X[0])
samples = X.shape[0]
train_len = 25
input_len = samples - train_len
I = np.zeros((samples - train_len, train_len, features))

(200000, 25)


In [30]:

for i in range(input_len):
    temp = np.zeros((train_len, features))
    for j in range(i, i + train_len):
        temp[j-i] = X[j]
    I[i] = temp

print(I.shape);

(199975, 25, 25)


In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(I, Y[25:200000], test_size = 0.2)
X_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [ ]:



from tensorflow.keras.layers import Add
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, GRU, MultiHeadAttention, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau

def create_complex_model():
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))


    cnn_out = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(input_layer)
    residual = cnn_out
    cnn_out = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(cnn_out)
    cnn_out = Add()([cnn_out, residual])
    cnn_out = MaxPooling1D(pool_size=2)(cnn_out)
    cnn_out = Dropout(0.3)(cnn_out)


    gru_out = Bidirectional(GRU(256, return_sequences=True, kernel_regularizer='l2'))(cnn_out)
    gru_out = Dropout(0.3)(gru_out)
    



    attention_out = MultiHeadAttention(num_heads=16, key_dim=32)(gru_out, gru_out)


    global_avg_pooling = GlobalAveragePooling1D()(attention_out)
    global_max_pooling = GlobalMaxPooling1D()(attention_out)
    concatenated = concatenate([global_avg_pooling, global_max_pooling])


    dense_out = Dense(256, activation='swish', kernel_regularizer='l2')(concatenated)
    dense_out = Dropout(0.3)(dense_out)
    dense_out = Dense(256, activation='swish', kernel_regularizer='l2')(dense_out)
    dense_out = Dropout(0.3)(dense_out)


    output_layer = Dense(1, activation='sigmoid')(dense_out)

    model = Model(inputs=input_layer, outputs=output_layer)


    model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=0.001, weight_decay=1e-5), metrics=['accuracy'])

    return model

model = create_complex_model()
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

tracker = EmissionsTracker()
tracker.start()

history = model.fit(X_train, Y_train, epochs=10, validation_split=0.2, verbose=1, callbacks=[lr_schedule])
model.summary()

tracker.stop()








[codecarbon INFO @ 11:07:04] [setup] RAM Tracking...
[codecarbon INFO @ 11:07:04] [setup] CPU Tracking...
[codecarbon WARNING @ 11:07:04] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 11:07:06] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 11:07:06] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 11:07:06] [setup] GPU Tracking...
[codecarbon INFO @ 11:07:06] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:07:06] >>> Tracker's metadata:
[codecarbon INFO @ 11:07:06]   Platform system: Linux-6.6.56+-x86_64-with-glibc2.35
[codecarbon INFO @ 11:07:06]   Python version: 3.10.12
[codecarbon INFO @ 11:07:06]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 11:07:06]   Available RAM : 31.351 GB
[codecarbon INFO @ 11:07:06]   CPU count: 4
[codeca

Epoch 1/10
 781/4000 ━━━━━━━━━━━━━━━━━━━━ 41s 13ms/step - accuracy: 0.8877 - loss: 2.2425

[codecarbon INFO @ 11:07:24] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:07:24] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:07:24] Energy consumed for all GPUs : 0.000364 kWh. Total GPU Power : 87.40610238603311 W
[codecarbon INFO @ 11:07:24] 0.000591 kWh of electricity used since the beginning.


1951/4000 ━━━━━━━━━━━━━━━━━━━━ 26s 13ms/step - accuracy: 0.9123 - loss: 1.2129

[codecarbon INFO @ 11:07:39] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:07:39] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:07:39] Energy consumed for all GPUs : 0.000784 kWh. Total GPU Power : 100.75336799004785 W
[codecarbon INFO @ 11:07:39] 0.001236 kWh of electricity used since the beginning.


...

4000/4000 ━━━━━━━━━━━━━━━━━━━━ 55s 14ms/step - accuracy: 0.9668 - loss: 0.1283 - val_accuracy: 0.9669 - val_loss: 0.1049 - learning_rate: 0.0010
Epoch 6/10
 369/4000 ━━━━━━━━━━━━━━━━━━━━ 47s 13ms/step - accuracy: 0.9724 - loss: 0.1110

[codecarbon INFO @ 11:11:54] Energy consumed for RAM : 0.000930 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:11:54] Energy consumed for all CPUs : 0.003363 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:11:54] Energy consumed for all GPUs : 0.007642 kWh. Total GPU Power : 95.44383446637289 W
[codecarbon INFO @ 11:11:54] 0.011935 kWh of electricity used since the beginning.


1529/4000 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - accuracy: 0.9705 - loss: 0.1235

[codecarbon INFO @ 11:12:09] Energy consumed for RAM : 0.000979 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:12:09] Energy consumed for all CPUs : 0.003540 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:12:09] Energy consumed for all GPUs : 0.008041 kWh. Total GPU Power : 95.79983694849145 W
[codecarbon INFO @ 11:12:09] 0.012560 kWh of electricity used since the beginning.


2522/4000 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9683 - loss: 0.1770

In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.savefig('Model Accuracy.png')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model  Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('Model Loss.png')
plt.show()

In [ ]:
predict = model.predict(X_test, verbose=1)

tp = 0
tn = 0
fp = 0
fn = 0
predictn = predict.flatten().round()
predictn = predictn.tolist()
Y_testn = Y_test.tolist()
for i in range(len(Y_testn)):
  if predictn[i]==1 and Y_testn[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_testn[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_testn[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_testn[i]==0:
    fn+=1

In [ ]:

to_heat_map =[[tn,fp],[fn,tp]]
to_heat_map = pd.DataFrame(to_heat_map, index = ["Attack","Normal"],columns = ["Attack","Normal"])
ax = sns.heatmap(to_heat_map,annot=True, fmt="d")

X_test = np.asarray(X_test, dtype=np.float32)
Y_test = np.asarray(Y_test, dtype=np.float32)
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

predictions_proba = model.predict(X_test)
predictions = (predictions_proba > 0.5).astype(int)

accuracy = accuracy_score(Y_test, predictions)

error_rate = 1 - accuracy

precision = precision_score(Y_test, predictions)

recall = recall_score(Y_test, predictions)

f1 = f1_score(Y_test, predictions)

auc = roc_auc_score(Y_test, predictions_proba)

print(f"Error Rate: {error_rate:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")


In [ ]:
import time
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
print(end_time - start_time)
print(f"Inference time per batch: {(end_time - start_time)/len(X_test):.6f} seconds")
